# Interpolating ICON-CH2-EPS Temperature Forecasts to Pressure Levels

This notebook demonstrates the full workflow for accessing ICON-CH2-EPS temperature forecasts and performing vertical interpolation from model levels to pressure levels. The data is provided by MeteoSwiss as part of Switzerland’s [Open Government Data (OGD) initiative](https://www.meteoswiss.admin.ch/services-and-publications/service/open-data.html).

The core functionality is powered by the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — developed in-house to simplify working with numerical weather model data. It includes the [ogd_api](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html) module for data access, along with utilities for applying temporal and spatial operators to model output, such as horizontal or vertical interpolation.

The ICON-CH1/2-EPS data is typically provided on terrain-following vertical model levels, which do not correspond directly to atmospheric pressure levels. However, for consistent comparison and analysis, it is often necessary to interpolate forecast data to these pressure levels. The [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library includes the function [interpolate_k2p](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.operators.vertical_interpolation.html#meteodatalab.operators.vertical_interpolation.interpolate_k2p), which interpolates a field from model levels to pressure coordinates.

---

## 🔍 **What You’ll Do in This Notebook**

 🛰️  **Retrieve**  
    Fetch deterministic ICON-CH2-EPS forecast data (e.g., temperature (`T`) and pressure (`P`)) using meteodata-lab's [ogd_api](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html) module.

 📈  **Interpolate to pressure levels**  
    Interpolate ICON-CH2-EPS forecast data to pressure levels using meteodata-lab's [vertical_interpolation](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.operators.vertical_interpolation.html#) module.

---

## Retrieving Forecasts
In this first part, we retrieve deterministic ICON-CH2-EPS temperature and pressure forecast data. To access this data, we use the [ogd_api](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html) module from the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — a convenient interface for accessing numerical weather forecasts via the [STAC (SpatioTemporal Asset Catalog) API](https://data.geo.admin.ch/api/stac/static/spec/v1/apitransactional.html#tag/Data/operation/getAsset), which provides structured access to Switzerland’s open geospatial data.

#### 📁  Browsing the STAC Catalog (Optional)

If you'd like to explore the ICON-CH1/2-EPS forecast datasets interactively before writing code, you can browse them directly in the STAC catalog:

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH1-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch1?.language=en)

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH2-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch2?.language=en)


Below is a screenshot of the ICON-CH2-EPS collection as seen in the STAC browser interface.


![browser-ch2-resized.PNG](./images/browser-ch2-resized.PNG)

### Creating Requests
To retrieve the forecast data, we first define API requests using the [ogd_api.Request](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html#meteodatalab.ogd_api.Request) class. In this example, we create two requests: one to retrieve the temperature field (`T`), which we aim to interpolate, and another to obtain the pressure field (`P`) on model levels, required for the interpolation.

>⏰ **Forecast Availability**: Forecast data will typically be available a couple of hours after the reference time — due to the model runtime and subsequent upload time. The data remains accessible for 24 hours after upload.

In [2]:
from datetime import datetime, timezone
from meteodatalab import ogd_api

# Set default date of today midnight in UTC
today_midnight_utc = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)

param_list = ["T", "P"]
req_list = []

for param in param_list:
    req = ogd_api.Request(
        collection="ogd-forecasting-icon-ch2",
        variable=param,
        reference_datetime=today_midnight_utc,
        perturbed=False,
        horizon=f"P0DT0H",
    )
    req_list.append((param,req))


Each argument in the request serves the following purpose:

| Argument             | Description |
|----------------------|-------------|
| `collection`         | Forecast collection to use (e.g., `ogd-forecasting-icon-ch2` for ICON-CH2-EPS). |
| `variable`           | Meteorological variable of interest (`T` = temperature and `P` = pressure). |
| `reference_datetime` | Initialization time of the forecast in **UTC**, provided as either:<br>- [datetime.datetime](https://docs.python.org/3/library/datetime.html#datetime-objects) object (e.g.,<br> &nbsp; `datetime.datetime(2025, 5, 22, 9, 0, 0, tzinfo=datetime.timezone.utc)`) <br>- [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Combined_date_and_time_representations) date string (e.g., `"2025-05-22T09:00:00Z"`)|
| `perturbed`          | If `True`, retrieves ensemble forecast members; if `False`, returns the deterministic forecast. |
| `horizon`            | Forecast lead time, provided as either:<br>– [datetime.timedelta](https://docs.python.org/3/library/datetime.html#timedelta-objects) object (e.g., `datetime.timedelta(hours=0)`) <br>– [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Durations) duration string (e.g., `"P0DT0H"`)|

### Retrieving Data
We now send our list of requests to the API and retrieve the resulting datasets using the [get_from_ogd()](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html#meteodatalab.ogd_api.get_from_ogd) function.
Each response is returned as an **[xarray.DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html)**, which is efficient for handling multi-dimensional data.

> 💡 **Tip**: Use temporary caching with earthkit-data to skip repeated downloads — it's auto-cleaned after the session.
> *For more details, see the [earthkit-data caching docs](https://earthkit-data.readthedocs.io/en/latest/examples/cache.html)*.

> 💡 **Hint**: If you get an error message containing `HTTPError: 403 Client Error: Forbidden for url`, you may be trying to retrieve data older than 24h hours! Please adjust your requests.

In [3]:
from earthkit.data import config
config.set("cache-policy", "temporary")

da_dict = {}
for param, req in req_list:
    da = ogd_api.get_from_ogd(req)
    da_dict[param]= da

The result of each `ogd_api.get_from_ogd()` call is an `xarray.DataArray` with the following structure:

**Dimensions**
- `eps` (ensemble members): 1 member (for deterministic data)
- `ref_time`: forecast init time (UTC)
- `lead_time`: time offset from `ref_time`
- `z`: <span style=\"color: red\"> **Vertical model levels (80)** </span><br>
- `cell`: spatial grid points (~280k for ICON-CH2-EPS)

**Coordinates**
- `lon` / `lat`: longitude & latitude per `cell`
- `valid_time`: `ref_time + lead_time`
        
**Attributes**
- `parameter`: variable info (e.g., name, units)
- `vcoord_type`: vertical coordinate type (e.g., `model_level`)
- `geography`: grid structure metadata

In [4]:
da_dict["T"]

<xarray.DataArray (eps: 1, ref_time: 1, lead_time: 1, z: 80, cell: 283876)> Size: 91MB
array([[[[[219.68489, 219.67256, 219.66817, ..., 219.70381, 219.68282,
           219.65633],
          [218.61911, 218.60751, 218.60092, ..., 221.14609, 221.13278,
           221.13266],
          [217.80774, 217.82715, 217.82092, ..., 221.1836 , 221.20117,
           221.2052 ],
          ...,
          [287.36298, 287.35028, 287.36298, ..., 277.94208, 277.98407,
           277.2741 ],
          [287.18143, 287.1414 , 287.1785 , ..., 277.83475, 278.0164 ,
           276.4832 ],
          [287.0555 , 286.9598 , 287.02133, ..., 277.36313, 277.72153,
           276.3475 ]]]]], dtype=float32)
Coordinates:
  * eps         (eps) int64 8B 0
  * ref_time    (ref_time) datetime64[ns] 8B 2025-05-23
  * lead_time   (lead_time) timedelta64[ns] 8B 00:00:00
  * z           (z) int64 640B 1 2 3 4 5 6 7 8 9 ... 72 73 74 75 76 77 78 79 80
    lon         (cell) float32 1MB -0.1544 -0.1637 -0.1915 ... 12.82 12.83 12.81
    lat         (cell) float32 1MB 46.97 46.98 46.98 47.13 ... 50.08 50.06 50.05
    valid_time  (ref_time, lead_time) datetime64[ns] 8B 2025-05-23
Dimensions without coordinates: cell
Attributes:
    metadata:     <earthkit.data.core.metadata.WrappedMetadata object at 0x70...
    parameter:    {'centre': 'lssw', 'paramId': 500014, 'shortName': 'T', 'un...
    geography:    {'gridType': 'unstructured_grid', 'bitmapPresent': 0}
    vref:         geo
    vcoord_type:  model_level
    origin_z:     0.0

> &#x1F525; **Expert users:** If you are interested in detailed meta data information, i.e. the GRIB encodings, you can retrieve it from the `metadata` attribute.
Please refer to the [earthkit-data metadata documentation](https://earthkit-data.readthedocs.io/en/latest/examples/metadata.html).

💡 To prepare for interpolation, let's inspect the initial vertical coordinate, the **model levels**:

In [5]:
da_dict["T"].coords["z"].values

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80])

#### Model levels

As shown in the output above, the parameter `T` includes **80 model levels**. These model levels represent how the atmosphere is divided from the Earth's surface up to the top of the model domain. In the numerical weather model ICON the 80 model levels correspond to the so-called **full levels** and are numbered top down. They are following the terrain and gradually change into levels of constant height as the distance from the surface increases (see the picture below). For more information about model levels, refer to the [model grid documentation](https://opendatadocs.meteoswiss.ch/e-forecast-data/e2-e3-numerical-weather-forecasting-model#vertical-grid).

<div style="text-align: center;">
  <img src="./images/VerticalLayers.png" width="50%">
</div>

### Interpolation from Model to Pressure Levels

Forecast data is typically stored on model (or hybrid) levels, which vary with the atmospheric state. For comparison, plotting, or diagnostics, it's often necessary to interpolate this data to **pressure levels**. The [`interpolate_k2p`](https://meteoswiss.github.io/meteodata-lab/) function from the **meteodata-lab** toolbox performs this transformation.

---
#### How `interpolate_k2p` Works

To compute the interpolated value $f(p_0)$ at a target pressure $p_0$, the function `interpolate_k2p`:

1. **Identifies two adjacent model levels** $k$ and $k-1$ such that:
   - $p_1 = \text{pressure}[k-1]$ is just **above** $p_0$
   - $p_2 = \text{pressure}[k]$ is just **below** $p_0$

2. **Retrieves the field values** at those levels:
   - $f_1 = \text{field}[k-1]$
   - $f_2 = \text{field}[k]$

3. **Computes the interpolation ratio** $r$ depending on the selected method:

    - **Linear Interpolation (`linear_in_p`)**
    $$
    r = \frac{p_0 - p_1}{p_2 - p_1}, \quad f(p_0) = (1 - r) f_1 + r f_2
    $$

    - **Linear Interpolation w.r.t Log-Pressure (`linear_in_lnp`)**
    $$
    r = \frac{\ln(p_0) - \ln(p_1)}{\ln(p_2) - \ln(p_1)}, \quad f(p_0) = (1 - r) f_1 + r f_2
    $$

    - **Nearest to Surface (`nearest_sfc`)**
    $$
    f(p_0) =
    \begin{cases}
    f_1 & \text{if } |p_0 - p_1| < |p_0 - p_2| \\
    f_2 & \text{otherwise}
    \end{cases}
    $$

This logic is applied point-wise across the full 3D field.

#### Comparison of Interpolation Methods

Each interpolation method produces slightly different results. For example, if you have temperature values at model levels *k* and *k-1* and want to estimate the value at 600 hPa, the result will depend on the method used. The following example illustrates how each of the three methods generates different outputs.

<div style="text-align: center;">
  <img src="./images/interpolation.png" width="50%">
</div>

#### Implementation
The choice of interpolation method depends on the relationship between the variable's vertical gradient and the pressure profile. In this example, we use logarithmic interpolation (`linear_in_lnp`) for temperature data, which is appropriate for variables that vary approximately linearly with the logarithm of pressure.

The `interpolate_k2p` function requires the following arguments:
- `field`: parameter to be interpolated
- `mode`: interpolation method (`"linear_in_p"`, `"linear_in_lnp"` or `"nearest_sfc"`)
- `p_field`: pressure field on k levels in Pa
- `p_tc_values`: target pressure levels
- `p_tc_units`: the unit of the target pressure levels (`"Pa"` or `"hPa"`)

In [43]:
from meteodatalab.operators.vertical_interpolation import interpolate_k2p
import numpy as np

target_levels = [500, 550, 600, 650, 700, 750, 800, 850]

T_int = interpolate_k2p(
    field=da_dict["T"],
    mode="linear_in_lnp",
    p_field=da_dict["P"],
    p_tc_values=target_levels,
    p_tc_units='hPa'
    )

### Pressure level inspection
After interpolation, the parameter now has updated vertical coordinates::
- `z`: 50000 - 85000 Pa (vertical levels)
- `vcoord_type`: pressure (type of vertical coordinates)

In [14]:
coords = T_int.coords["z"]
attrs = T_int.attrs["vcoord_type"]
print(f"\033[1mz \033[0m(vertical model levels): {coords.values}")
print(f"\033[1mvcoord_type \033[0m(type of vertical coordinate): {attrs}")

z (vertical model levels): [50000. 55000. 60000. 65000. 70000. 75000. 80000. 85000.]
vcoord_type (type of vertical coordinate): pressure
